In [1]:
import os
import shutil
import math
from Crypto.Cipher import AES, DES, DES3, ARC4
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad

def generate_keys(keys_per_algorithm):
    keys = {}
    for algo in ['AES', 'DES', 'DES3', 'ARC4']:
        key_list = []
        for _ in range(keys_per_algorithm):
            if algo == 'AES':
                key_list.append(get_random_bytes(16))
            elif algo == 'DES':
                key_list.append(get_random_bytes(8))
            elif algo == 'DES3':
                key_list.append(DES3.adjust_key_parity(get_random_bytes(24)))
            elif algo == 'ARC4':
                key_list.append(get_random_bytes(16))
        keys[algo] = key_list
    return keys

def encrypt_cbc(algo, key, data):
    if algo == 'AES':
        iv = get_random_bytes(16)
        cipher = AES.new(key, AES.MODE_CBC, iv)
        padded_data = pad(data, AES.block_size)
        return iv + cipher.encrypt(padded_data)
    elif algo == 'DES':
        iv = get_random_bytes(8)
        cipher = DES.new(key, DES.MODE_CBC, iv)
        padded_data = pad(data, DES.block_size)
        return iv + cipher.encrypt(padded_data)
    elif algo == 'DES3':
        iv = get_random_bytes(8)
        cipher = DES3.new(key, DES.MODE_CBC, iv)
        padded_data = pad(data, DES3.block_size)
        return iv + cipher.encrypt(padded_data)
    elif algo == 'ARC4':
        cipher = ARC4.new(key)
        return cipher.encrypt(data)
    else:
        raise ValueError(f"Unsupported algorithm: {algo}")

def encrypt_files_by_format_cbc(base_dir, save_dir, keys_per_algorithm=3):
    formats = ['bmp', 'csv', 'html', 'pyth', 'txt', 'wav']
    algorithms = ['AES', 'DES', 'DES3', 'ARC4', 'NONE']

    keys = generate_keys(keys_per_algorithm)

    for folder in formats:
        input_dir = os.path.join(base_dir, folder)
        if not os.path.exists(input_dir):
            continue

        file_list = sorted([f for f in os.listdir(input_dir) if os.path.isfile(os.path.join(input_dir, f))])
        total_files = len(file_list)
        files_per_algo = math.ceil(total_files / len(algorithms))

        os.makedirs(save_dir, exist_ok=True)

        for i, algo in enumerate(algorithms):
            start_idx = i * files_per_algo
            end_idx = min((i + 1) * files_per_algo, total_files)
            selected_files = file_list[start_idx:end_idx]

            output_dir = os.path.join(save_dir, f"{folder}_cbc", f"{algo}")
            os.makedirs(output_dir, exist_ok=True)

            for j, file_name in enumerate(selected_files):
                input_path = os.path.join(input_dir, file_name)
                output_path = os.path.join(output_dir, file_name + ('' if algo == 'NONE' else '.enc'))

                with open(input_path, 'rb') as f:
                    plain_text = f.read()

                if algo == 'NONE':
                    shutil.copy2(input_path, output_path)
                else:
                    key_index = j % keys_per_algorithm
                    key = keys[algo][key_index]
                    cipher_text = encrypt_cbc(algo, key, plain_text)
                    with open(output_path, 'wb') as f_encrypted:
                        f_encrypted.write(cipher_text)

    print(f"All files encrypted (CBC mode) using {keys_per_algorithm} keys per algorithm.")

In [3]:
encrypt_files_by_format_cbc('files/1000_files', 'encrypted_cbc/encrypted_cbc_1000/encrypted_cbc_1000_1key', 1)

All files encrypted (CBC mode) using 1 keys per algorithm.


In [4]:
encrypt_files_by_format_cbc('files/1000_files', 'encrypted_cbc/encrypted_cbc_1000/encrypted_cbc_1000_3keys', 3)

All files encrypted (CBC mode) using 3 keys per algorithm.


In [5]:
encrypt_files_by_format_cbc('files/1000_files', 'encrypted_cbc/encrypted_cbc_1000/encrypted_cbc_1000_6keys', 6)

All files encrypted (CBC mode) using 6 keys per algorithm.


In [6]:
encrypt_files_by_format_cbc('files/1800_files', 'encrypted_cbc/encrypted_cbc_1800/encrypted_cbc_1800_1key', 1)

All files encrypted (CBC mode) using 1 keys per algorithm.


In [7]:
encrypt_files_by_format_cbc('files/1800_files', 'encrypted_cbc/encrypted_cbc_1800/encrypted_cbc_1800_3keys', 3)

All files encrypted (CBC mode) using 3 keys per algorithm.


In [8]:
encrypt_files_by_format_cbc('files/1800_files', 'encrypted_cbc/encrypted_cbc_1800/encrypted_cbc_1800_6keys', 6)

All files encrypted (CBC mode) using 6 keys per algorithm.


In [9]:
encrypt_files_by_format_cbc('files/3000_files', 'encrypted_cbc/encrypted_cbc_3000/encrypted_cbc_3000_1key', 1)

All files encrypted (CBC mode) using 1 keys per algorithm.


In [10]:
encrypt_files_by_format_cbc('files/3000_files', 'encrypted_cbc/encrypted_cbc_3000/encrypted_cbc_3000_3keys', 3)

All files encrypted (CBC mode) using 3 keys per algorithm.


In [11]:
encrypt_files_by_format_cbc('files/3000_files', 'encrypted_cbc/encrypted_cbc_3000/encrypted_cbc_3000_6keys', 6)

All files encrypted (CBC mode) using 6 keys per algorithm.
